## Preprocessing and Naive-Bayes

In [4]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np
import matplotlib.pyplot as plt

### Pre-Processing to remove unnecessary columns and restricting the values to analyzable ranges.

In [5]:
capture_34_dataset = "D:/Masters/SEM-2/es/SVM_v1/myenv/conn.log.labeled"
df_34 = pd.read_table(filepath_or_buffer=capture_34_dataset, skiprows=10, nrows=100000)
df_34.columns=['ts',
              'uid',
              'id.orig_h',
              'id.orig_p',
              'id.resp_h',
              'id.resp_p',
              'proto',
              'service',
              'duration',
              'orig_bytes',
              'resp_bytes',
              'conn_state',
              'local_orig',
              'local_resp',
              'missed_bytes',
              'history',
              'orig_pkts',
              'orig_ip_bytes',
              'resp_pkts',
              'resp_ip_bytes',
              'label']

df_34.drop(df_34.tail(1).index,inplace=True)

frames=[df_34]
df_c=pd.concat(frames)

In [6]:
df_c

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,1.545404e+09,CDrkrSobGYxHhYfth,192.168.1.195,41040,185.244.25.235,80,tcp,http,1.477656,149,...,SF,-,-,2896,ShADadttcfF,94,5525,96,139044,- Benign -
1,1.545404e+09,CTWZQf2oJSvq6zmPAc,192.168.1.195,41042,185.244.25.235,80,tcp,-,3.147116,0,...,S0,-,-,0,S,3,180,0,0,- Benign -
2,1.545404e+09,CvyynC4Sabj9BNXFRi,192.168.1.195,41042,185.244.25.235,80,tcp,http,1.305004,151,...,SF,-,-,5792,ShADadttcfF,96,5699,92,133140,- Benign -
3,1.545404e+09,CWYyiA2sgRijwk2jEd,192.168.1.195,41044,185.244.25.235,80,tcp,http,1.004605,148,...,SF,-,-,2896,ShADadtcfF,67,4148,65,92453,- Benign -
4,1.545404e+09,CYttPy2pqOIcen7UDh,192.168.1.195,41046,185.244.25.235,80,tcp,http,4.129647,148,...,SF,-,-,5792,ShADadtcfF,75,4412,69,97107,- Benign -
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23136,1.545490e+09,CRPM7qYJN4QudEl47,192.168.1.195,57086,185.244.25.235,6667,tcp,irc,33.498912,62,...,S3,-,-,0,ShAdDaf,7,434,6,589,- Malicious C&C
23137,1.545490e+09,C2F17zSUnGOcWzBa7,192.168.1.195,57110,185.244.25.235,6667,tcp,irc,32.840994,62,...,S3,-,-,0,ShAdDaf,7,434,6,589,- Malicious C&C
23138,1.545490e+09,C93P4z4k5IRJD1rXJg,192.168.1.195,57092,185.244.25.235,6667,tcp,irc,36.290833,62,...,S3,-,-,0,ShAdDaf,10,606,7,632,- Malicious C&C
23139,1.545490e+09,CXLZ3A2QY5E8weqpDk,192.168.1.195,123,147.251.48.140,123,udp,-,-,-,...,S0,-,-,0,D,1,76,0,0,- Benign -


In [7]:
df_c['label'].value_counts()

label
-   Malicious   DDoS                         14394
-   Malicious   C&C                           6706
-   Benign   -                                1919
-   Malicious   PartOfAHorizontalPortScan      122
Name: count, dtype: int64

In [8]:
df_c.loc[(df_c.label == '-   Malicious   PartOfAHorizontalPortScan'), 'label'] = 'PartOfAHorizontalPortScan'
df_c.loc[(df_c.label == '(empty)   Malicious   PartOfAHorizontalPortScan'), 'label'] = 'PartOfAHorizontalPortScan'
df_c.loc[(df_c.label == '-   Malicious   Okiru'), 'label'] = 'Okiru'
df_c.loc[(df_c.label == '(empty)   Malicious   Okiru'), 'label'] = 'Okiru'
df_c.loc[(df_c.label == '-   Benign   -'), 'label'] = 'Benign'
df_c.loc[(df_c.label == '(empty)   Benign   -'), 'label'] = 'Benign'
df_c.loc[(df_c.label == '-   Malicious   DDoS'), 'label'] = 'DDoS'
df_c.loc[(df_c.label == '-   Malicious   C&C'), 'label'] = 'C&C'
df_c.loc[(df_c.label == '(empty)   Malicious   C&C'), 'label'] = 'C&C'
df_c.loc[(df_c.label == '-   Malicious   Attack'), 'label'] = 'Attack'
df_c.loc[(df_c.label == '(empty)   Malicious   Attack'), 'label'] = 'Attack'
df_c.loc[(df_c.label == '-   Malicious   C&C-HeartBeat'), 'label'] = 'C&C-HeartBeat'
df_c.loc[(df_c.label == '(empty)   Malicious   C&C-HeartBeat'), 'label'] = 'C&C-HeartBeat'
df_c.loc[(df_c.label == '-   Malicious   C&C-FileDownload'), 'label'] = 'C&C-FileDownload'
df_c.loc[(df_c.label == '-   Malicious   C&C-Torii'), 'label'] = 'C&C-Torii'
df_c.loc[(df_c.label == '-   Malicious   C&C-HeartBeat-FileDownload'), 'label'] = 'C&C-HeartBeat-FileDownload'
df_c.loc[(df_c.label == '-   Malicious   FileDownload'), 'label'] = 'FileDownload'
df_c.loc[(df_c.label == '-   Malicious   C&C-Mirai'), 'label'] = 'C&C-Mirai'
df_c.loc[(df_c.label == '-   Malicious   Okiru-Attack'), 'label'] = 'Okiru-Attack'

In [9]:
df_c['label'].value_counts()

label
DDoS                         14394
C&C                           6706
Benign                        1919
PartOfAHorizontalPortScan      122
Name: count, dtype: int64

In [10]:
df_c = df_c.drop(columns=['ts','uid','id.orig_h','id.orig_p','id.resp_h','id.resp_p', 'service','local_orig','local_resp','history'])

In [11]:
df_c = pd.get_dummies(df_c, columns=['proto'])
df_c = pd.get_dummies(df_c, columns=['conn_state'])

In [12]:
df_c

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_tcp,proto_udp,conn_state_OTH,conn_state_RSTR,conn_state_S0,conn_state_S1,conn_state_S3,conn_state_SF
0,1.477656,149,128252,2896,94,5525,96,139044,Benign,True,False,False,False,False,False,False,True
1,3.147116,0,0,0,3,180,0,0,Benign,True,False,False,False,True,False,False,False
2,1.305004,151,128348,5792,96,5699,92,133140,Benign,True,False,False,False,False,False,False,True
3,1.004605,148,91961,2896,67,4148,65,92453,Benign,True,False,False,False,False,False,False,True
4,4.129647,148,99303,5792,75,4412,69,97107,Benign,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23136,33.498912,62,269,0,7,434,6,589,C&C,True,False,False,False,False,False,True,False
23137,32.840994,62,269,0,7,434,6,589,C&C,True,False,False,False,False,False,True,False
23138,36.290833,62,260,0,10,606,7,632,C&C,True,False,False,False,False,False,True,False
23139,-,-,-,0,1,76,0,0,Benign,False,True,False,False,True,False,False,False


In [13]:
df_c['duration'] = df_c['duration'].str.replace('-','0')
df_c['orig_bytes'] = df_c['orig_bytes'].str.replace('-','0')
df_c['resp_bytes'] = df_c['resp_bytes'].str.replace('-','0')
df_c

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_tcp,proto_udp,conn_state_OTH,conn_state_RSTR,conn_state_S0,conn_state_S1,conn_state_S3,conn_state_SF
0,1.477656,149,128252,2896,94,5525,96,139044,Benign,True,False,False,False,False,False,False,True
1,3.147116,0,0,0,3,180,0,0,Benign,True,False,False,False,True,False,False,False
2,1.305004,151,128348,5792,96,5699,92,133140,Benign,True,False,False,False,False,False,False,True
3,1.004605,148,91961,2896,67,4148,65,92453,Benign,True,False,False,False,False,False,False,True
4,4.129647,148,99303,5792,75,4412,69,97107,Benign,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23136,33.498912,62,269,0,7,434,6,589,C&C,True,False,False,False,False,False,True,False
23137,32.840994,62,269,0,7,434,6,589,C&C,True,False,False,False,False,False,True,False
23138,36.290833,62,260,0,10,606,7,632,C&C,True,False,False,False,False,False,True,False
23139,0,0,0,0,1,76,0,0,Benign,False,True,False,False,True,False,False,False


In [14]:
df_c.fillna(-1,inplace=True)
check=df_c.isna().sum()
check

duration           0
orig_bytes         0
resp_bytes         0
missed_bytes       0
orig_pkts          0
orig_ip_bytes      0
resp_pkts          0
resp_ip_bytes      0
label              0
proto_tcp          0
proto_udp          0
conn_state_OTH     0
conn_state_RSTR    0
conn_state_S0      0
conn_state_S1      0
conn_state_S3      0
conn_state_SF      0
dtype: int64

In [15]:
df_c

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_tcp,proto_udp,conn_state_OTH,conn_state_RSTR,conn_state_S0,conn_state_S1,conn_state_S3,conn_state_SF
0,1.477656,149,128252,2896,94,5525,96,139044,Benign,True,False,False,False,False,False,False,True
1,3.147116,0,0,0,3,180,0,0,Benign,True,False,False,False,True,False,False,False
2,1.305004,151,128348,5792,96,5699,92,133140,Benign,True,False,False,False,False,False,False,True
3,1.004605,148,91961,2896,67,4148,65,92453,Benign,True,False,False,False,False,False,False,True
4,4.129647,148,99303,5792,75,4412,69,97107,Benign,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23136,33.498912,62,269,0,7,434,6,589,C&C,True,False,False,False,False,False,True,False
23137,32.840994,62,269,0,7,434,6,589,C&C,True,False,False,False,False,False,True,False
23138,36.290833,62,260,0,10,606,7,632,C&C,True,False,False,False,False,False,True,False
23139,0,0,0,0,1,76,0,0,Benign,False,True,False,False,True,False,False,False


In [16]:
df_c['label'].value_counts()

label
DDoS                         14394
C&C                           6706
Benign                        1919
PartOfAHorizontalPortScan      122
Name: count, dtype: int64

In [17]:
for col in df_c.columns: 
    print(col) 

duration
orig_bytes
resp_bytes
missed_bytes
orig_pkts
orig_ip_bytes
resp_pkts
resp_ip_bytes
label
proto_tcp
proto_udp
conn_state_OTH
conn_state_RSTR
conn_state_S0
conn_state_S1
conn_state_S3
conn_state_SF


In [18]:
X = df_c[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_RSTR', 'conn_state_S0', 'conn_state_S1', 'conn_state_S3', 'conn_state_SF']]
Y = df_c['label']

## SVM for IOT-23 Dataset with CTU-IoT-Malware-Capture-34-1 (Mirai)

In [19]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np

In [20]:
SVM_classifier = SVC()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)

In [21]:
start = time.time()
print('program start...')
print()

SVM_classifier = SVC(C=1.0, cache_size=1500, verbose=True).fit(X_train, Y_train)
print()
print(SVM_classifier.score(X_test, Y_test))
print()

y_pred = SVM_classifier.predict(X_test)
print(y_pred)
print()

end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

program start...

[LibSVM]
0.696046662346079

['DDoS' 'DDoS' 'DDoS' ... 'DDoS' 'DDoS' 'DDoS']

program end...

time cost: 
22.016252040863037 seconds


In [22]:
print("Classifiction Report :")
print(classification_report(Y_test, y_pred))

Classifiction Report :
                           precision    recall  f1-score   support

                   Benign       0.60      0.01      0.02       392
                      C&C       0.98      0.26      0.41      1338
                     DDoS       0.67      1.00      0.80      2877
PartOfAHorizontalPortScan       0.00      0.00      0.00        22

                 accuracy                           0.70      4629
                macro avg       0.56      0.32      0.31      4629
             weighted avg       0.75      0.70      0.62      4629

